In [1]:
from ultralytics import YOLO
import cv2
from queue import Queue
import threading
from lib.Models import Models
import time

In [2]:

class_names = ["dog", "cow"] # Nombres de las clases a detectar con yolo
dog_info = {}

model_general = YOLO('yolo11n.pt')  # Modelo para detectar personas y vehículos
models = Models(dog_info)  # Modelos para el procesamiento de los caracteristicas

filename = "video_dogs.mp4" # Video a procesar
output_video_filename = "resultado_deteccion.mp4" #Video con la detección y el procesado

intel_dog_data_csv_filename = "dog_intelligence.csv" # Archivo con la información de la inteligencia promedio de los perros en base a la raza
size_dog_data_csv_filename = "AKC_Breed_info.csv" # Archivo con la información del tamaño promedio de los perros en base a la raza



In [3]:
threads_number = 1


id_buffer = Queue()

pause_event = threading.Event()

#semaphore = threading.Semaphore(4)

pause_event.clear()


def frame_to_characteristics():
    while True:
        
        track_id, score = id_buffer.get()
        
        if track_id is None: pause_event.wait()

        pause_event.clear()
        threading.Thread(target=models.set_characteristics, args=(track_id,)).start()
        #print(f"ID: {track_id}, Score: {score}")
        

characteristics_thread = threading.Thread(target=frame_to_characteristics)
characteristics_thread.start()

In [4]:

results = model_general.track(source=filename, stream=True, verbose=False)
frame_count = 0

def set_track_id(image):
    list_characteristics = dog_info.get(track_id, [image, score, "Unknown", "Unknown", "Unknown", 0])
    list_characteristics[0] = image
    list_characteristics[1] = score
    dog_info[track_id] = list_characteristics

for frame_result in results:
    frame_count += 1
    frame = frame_result.orig_img
    time.sleep(0.05)

    # Imprimir fotograma actual y total de fotogramas
    #print(f'Procesando fotograma {frame_count}')
    
    for box in frame_result.boxes:
        
        x1, y1, x2, y2 = map(int, box.xyxy[0])
        score = box.conf[0].item()
        label = int(box.cls[0])
        track_id = int(box.id[0]) if box.id is not None else -1
        
        #Validación de label  dog = 16  y  Umbral de confianza
        if score>0.6 and label==16 : 
            if (dog_info.get(track_id, [0, 0, "Unknown", "Unknown", "Unknown", 0])[1] < score):
                id_buffer.put([track_id, score])
                pause_event.set()
                set_track_id(frame[y1:y2, x1:x2])


        if track_id in dog_info:
            cv2.rectangle(frame, (x1,  y1), (x2, y2), (0, 255, 0), 2)
            #cv2.putText(frame, f'ID: {track_id}', (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 0), 2)
            #cv2.putText(frame,  f"Count: {dog_info[track_id][5]}", (x1, y1 - 50), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 255), 2)
            cv2.putText(frame,  f"Emotion: {dog_info[track_id][4]}", (x1, y1 - 50), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 255), 2)
            cv2.putText(frame,  f"Color: {dog_info[track_id][2]}", (x1, y1 - 30), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 255), 2)
            cv2.putText(frame,  f"Breed: {dog_info[track_id][3]}", (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 255), 2)

    
    cv2.imshow('Detección en proceso', frame)

    # Esperar brevemente para permitir visualización, y detener si se presiona 'q'
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
    
cv2.destroyAllWindows()

[13.74733533 11.84318687 26.1749099 ]
[13.54257691 11.72596225 26.26787041]
[32.58939243 31.73655378 41.09462151]
[15.75332376 13.69808377 27.48080644]
[124.08535855 127.0555489  110.11133724]
[34.15616207 33.43866066 43.07118739]
[11.06468872  7.69464981 18.59007782]
[166.97324083 135.7152296  105.59795177]
[12.42526551  9.02996087 20.79631079]
[117.67321867 103.51867322 106.05307125]
[31.63876652 31.08193833 40.06079295]
[133.84295393 114.28482385  86.63821138]
[113.32821724 128.01574183 103.78787879]
[60.30076035 54.68459589 58.34947902]
[37.09356899 30.695678   43.54284866]
[50.24209684 47.05602241 52.67386955]
[30.6286123  26.87167382 37.84763948]
[32.3494246  28.40352713 39.29591989]
[65.14396671 57.67961165 62.64687933]
[133.13727094 127.76537958 125.60111257]
[141.72147455 139.47454652 121.53686366]
